In [ ]:
from postprocessing import *
import scipy as scp
from scipy import stats
from natsort import natsorted
import glob
import os

In [ ]:
mpl.rcParams['figure.dpi']= 200

### physics approach

In [ ]:
n_w = 10 ** 1.7
T_w = 10 ** 3.61

n_h = 0.1
T_h =  n_w * T_w / n_h

In [ ]:
np.round(n_w, -1), np.round(T_w, -2) ### warm phase

In [ ]:
np.round(n_h, 2), np.round(T_h, -4) ### hot phase

In [ ]:
x = np.linspace(0, 2000, 2001)
pdf = stats.norm.pdf(x, loc=1300, scale=10)
n = (pdf / pdf.max()) * n_w + n_h
T = n_w * T_w / n

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 4))
ax2 = ax1.twinx()

font_1 = {'color':  'blue'}
font_2 = {'color':  'red'}

ax1.plot(x, n, 'b-', lw=2, alpha=0.6, label='density')
ax2.plot(x, T, 'r-', lw=2, alpha=0.6, label='temperature')

ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.set_ylabel('number density', fontdict=font_1)
ax2.set_ylabel('temperature', fontdict=font_2)

### approach in coding

In [ ]:
import numpy as np
from scipy.stats import norm

def generate_gaussian_grid(loc, sigma, num_points, grid_min, grid_max):
    # Generate samples from the custom distribution
    
    u = np.linspace(0, 1, num_points) #np.random.rand(n_samples)
    
    # Apply the inverse transform
    x = norm.ppf(u, loc=loc, scale=sigma)
    
    return x#positions


Ncells = 256
BoxSize = 2000
NumberOfCells = Ncells ** 3
dx = BoxSize / Ncells
pos_first, pos_last = 0.5 * dx, BoxSize - 0.5 * dx

loc = 1200  # Mean of the Gaussian distribution
sigma = 5  # Standard deviation of the Gaussian distribution
num_points = 50  # Number of points to generate

gaussian_grid = generate_gaussian_grid(loc, sigma, num_points, pos_first, pos_last)
uniform_grid  = np.linspace(pos_first, pos_last, Ncells - num_points)

Grid1d = np.linspace(pos_first, pos_last, Ncells)
Grid1d_gaussian = np.sort(np.append(uniform_grid, gaussian_grid))
xx, yy, zz = np.meshgrid(Grid1d_gaussian, Grid1d, Grid1d)
Pos = np.zeros([NumberOfCells, 3])
Pos[:,0] = xx.reshape(NumberOfCells)
Pos[:,1] = yy.reshape(NumberOfCells)
Pos[:,2] = zz.reshape(NumberOfCells)
# center = np.array([0.5*BoxSize,0.5*BoxSize,0.5*BoxSize])

In [ ]:
fig, ax = plt.subplots()
plt.scatter(Pos[:,0], Pos[:,1], s=0.5)

plt.xlim(1100, 1300)
plt.ylim(1100, 1300)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Pos[:,1], Pos[:,2])
plt.xlim(1100, 1300)
plt.ylim(1100, 1300)

### result ....

In [ ]:
density = 10
mach = 8
jetpower = 38
start = 'wall'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
filename = "/IC.hdf5"
IC = h5py.File(simulation_directory +  "/IC.hdf5",'r')

In [ ]:
i_file = 1
filename = "snap_%03d.hdf5" % (i_file)
fig, ax = plt.subplots(figsize=(6,7))
plot_dens_vel(ax, output_directory + filename, fac=0.5, t0=0)
#plot_pressure(ax, output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
#plot_temp_vel(ax, output_directory + "snap_%03d.hdf5" % (0 + i_file), fac=0.5, t0=0)

In [ ]:
i_file = 0
snapshot = h5py.File(output_directory +  "snap_%03d.hdf5" % (i_file),'r')

In [ ]:
snapshot['PartType0/Density'][:] * rho_to_numdensity

In [ ]:
x = IC['PartType0/Coordinates'][:, 0]
y = IC['PartType0/Coordinates'][:, 1]
z = IC['PartType0/Coordinates'][:, 2]

# x = snapshot['PartType0/Coordinates'][:, 0]
# y = snapshot['PartType0/Coordinates'][:, 1]
# z = snapshot['PartType0/Coordinates'][:, 2]

In [ ]:
dy = np.diff(y)
print(dy)

In [ ]:
dy = np.append(dy, dy[-1])

In [ ]:
dy.shape[0] / 256 / 256

In [ ]:
data_3d = dy.reshape((256, 256, 256))

# Select the slice you want to plot in the x-y projection
slice_xy = data_3d[:,150,:]  # Change 100 to the desired z-slice index

# Plot the slice using imshow
plt.imshow(slice_xy, cmap='viridis')  # You can change the colormap if you prefer
plt.colorbar()  # Add a colorbar to indicate values
plt.title('Slice in X-Y Projection')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
np.unique(np.diff(y))

In [ ]:
plt.scatter(z, x, s=0.5)
plt.xlim(1100, 1300)
plt.ylim(1100, 1300)

In [ ]:
i_file = 0
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory + filename, "r")
snapshot = h5py.File(output_directory +  "snap_%03d.hdf5" % (i_file),'r')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
time = get_time_from_snap(snapshot)
masses = snapshot['PartType0/Masses'][:]
densities = snapshot['PartType0/Density'][:] * rho_to_numdensity
temperatures = get_temp(output_directory + filename, 5/3)

h = ax.hist2d(np.log10(densities), np.log10(temperatures), weights=masses, 
              bins=100, density=True, norm=mpl.colors.LogNorm(vmin=1e-2, vmax=5e5))
# ax.scatter(np.log10(densities), np.log10(temperatures))
ax.set_xlabel(r'log $n_H$')
ax.set_ylabel('log T')
ax.set_title("t=%.2f Myr"%(get_time_from_snap(snap_data) * unit_time_in_megayr))
ax.set_xlim(-2, 2)
ax.set_ylim(3.2 , 6.5)

# ax.axvline(2, c='black')
# ax.axhline(3.58, c='black')
ax.scatter(1.7, 3.61, c='lightblue', s=9, zorder=100)
ax.scatter(-1, 6.31, c='tab:red', s=9, zorder=100)

plt.colorbar(h[3], ax=ax)

# again trying to code it

In [ ]:
Volume_line = dx * dx * np.append(np.diff(Grid1d_gaussian), np.diff(Grid1d_gaussian)[-1])
Volume = np.broadcast_to(Volume_line, (len(Volume_line), len(Volume_line), len(Volume_line))).flatten()

In [ ]:
import sys
import numpy as np
import h5py
from scipy.stats import norm

def generate_gaussian_grid(loc, sigma, num_points, grid_min, grid_max):
    # Generate normally distributed points
    gaussian_points = np.random.normal(loc=loc, scale=sigma, size=num_points)

    # Clip points to ensure they fall within the grid bounds
    gaussian_points = np.clip(gaussian_points, grid_min, grid_max)

    # Sort the points
    gaussian_points.sort()

    return gaussian_points

#simulation_directory = str(sys.argv[1])
simulation_directory = '.'

""" simulation box parameters """
FloatType = np.float64
IntType = np.int32

# cgs unit
PROTONMASS = FloatType(1.67262178e-24)
BOLTZMANN = FloatType(1.38065e-16)
GRAVITY = FloatType(6.6738e-8)
PC = FloatType(3.085678e+18)
MYR = FloatType(3.15576e13)
MSOLAR = FloatType(1.989e+33)

UnitLength = PC # pc
UnitMass = MSOLAR # Msun
UnitVelocity = FloatType(1e5) # km/s
UnitTime = UnitLength/UnitVelocity
UnitDensity = UnitMass / UnitLength / UnitLength / UnitLength
mu = np.float64(0.6165)
print ("UnitTime_in_Myr = %.2f"%(UnitTime/MYR))

# simulation set up
GAMMA = 5./3.
BoxSize = FloatType(2000.0) # in code unit
IsoSoundspeed = FloatType(20.0) # in code unit
nH_ISM = 10.0 # in cm^-3
n_warm = 50   # cm^-3
n_hot  = 0.1  # cm^-3
T_warm = 4100 # K
T_hot  = 2e6  # K

RhoAve = nH_ISM*1.4*PROTONMASS/UnitDensity # in code unit
RhoHot = n_hot *1.4*PROTONMASS/UnitDensity # in code unit

In [ ]:
NumSnaps = IntType(600)
Ncells = IntType(156) #can do 64 if 128 is too slow
Ncells_extra = IntType(20)
NumberOfCells = IntType( Ncells * Ncells * (Ncells + Ncells_extra))
TimeMax = FloatType(30)#FloatType(1.0*BoxSize/IsoSoundspeed) # in code unit
print ("TimeMax (code unit) = ",TimeMax)

#BH parameters
BH_Hsml = FloatType(90) # r_jet=30
HalfOpeningAngle = FloatType(0)
vTargetJet = (BH_Hsml / 10.)**3
JetDensity = FloatType(1e-26) # g cm^-3
JetDensity_code_units = JetDensity / UnitDensity

""" set up initial conditions """
dx = BoxSize / FloatType(Ncells)
pos_first, pos_last = 0.5 * dx, BoxSize - 0.5 * dx

loc = 1500  # Mean of the Gaussian distribution
sigma = 5  # Standard deviation of the Gaussian distribution
num_points = Ncells_extra  # Number of points to generate

gaussian_grid = generate_gaussian_grid(loc, sigma, num_points, pos_first, pos_last)
# uniform_grid  = np.linspace(pos_first, pos_last, Ncells - num_points)
uniform_grid  = np.linspace(500, 1700, Ncells)

Grid1d = np.linspace(pos_first, pos_last, Ncells, dtype=FloatType)
Grid1d_gaussian = np.sort(np.append(uniform_grid, gaussian_grid))
xx, yy, zz = np.meshgrid(Grid1d_gaussian, Grid1d, Grid1d)
Pos = np.zeros([NumberOfCells, 3], dtype=FloatType)
Pos[:,0] = xx.reshape(NumberOfCells)
Pos[:,1] = yy.reshape(NumberOfCells)
Pos[:,2] = zz.reshape(NumberOfCells)
center = np.array([0.5*BoxSize,0.5*BoxSize,0.5*BoxSize])

In [ ]:
Pos.shape

In [ ]:
cell_volumes_line = dx * dx * np.append(np.diff(Grid1d_gaussian), np.diff(Grid1d_gaussian)[-1])
cell_volumes = np.zeros((Ncells, Ncells + Ncells_extra, Ncells), dtype=FloatType)

for i in range(Ncells):
    for j in range(Ncells):
        cell_volumes[i, :, j] = cell_volumes_line
        
cell_volumes = cell_volumes.reshape(NumberOfCells)

In [ ]:
#cell_volumes = np.zeros(len(Pos))  # Initialize with zeros
cell_volumes_line = dx * dx * np.append(np.diff(Grid1d_gaussian), np.diff(Grid1d_gaussian)[-1])
#cell_volumes = np.repeat(cell_volumes_line, len(cell_volumes_line) ** 2)
cell_volumes = np.broadcast_to(cell_volumes_line, (len(cell_volumes_line), len(cell_volumes_line), len(cell_volumes_line))).flatten()

In [ ]:
np.append(np.diff(Grid1d_gaussian), np.diff(Grid1d_gaussian)[-1])[0]

In [ ]:
dx

In [ ]:
plt.plot(np.unique(Pos[:, 0]), cell_volumes_line)
plt.axhline(dx**3)
plt.xlim(1450, 1550)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.scatter(Pos[:, 0], Pos[:, 1], c=cell_volumes, s=1) #, norm=mpl.colors.LogNorm(vmin=1e-8, vmax=5e1))
plt.xlim(1450,1550)
plt.ylim(1400,1500)

In [ ]:
plt.hist(cell_volumes, bins=20)

In [ ]:
!ls {output_directory}

In [ ]:
i_file = 0#n0_jet # skip snap 0
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    fig, ax = plt.subplots(2, 2, figsize=(8, 6.1))
    fig.tight_layout(w_pad=7.0, h_pad=3.0)
    plot_dens_vel(ax[0][0], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
    plot_temp_vel(ax[0][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
    plot_pressure(ax[1][0], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
    plot_jet_tracer(ax[1][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
    
    plt.savefig(figures_directory + f'4panels_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/4panels*.png'
ofilename = figures_directory + '/4panels-jet.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)